Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [48]:
from random import choices, randint

import lab9_lib

In [49]:
fitness = lab9_lib.make_problem(10)                                    #fitness is an instance of the class Problem (subclass of AbstractProblem)
for n in range(10):
    ind = choices([0, 1], k=50)                                        #ind is the genome
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")       #fitness(ind) calls the function __call__ which calculates the fitness value

print(fitness.calls)

10000110101110000011101110000111001100000001000110: 7.36%
10101001111100101101110100011010101000111010101101: 9.11%
01100100100001010001010000010001011010111001100001: 17.56%
00001011001101101011011011001111101110001000000001: 7.33%
00101111110100110101011111010101110111001011001000: 23.33%
10000111010001110100001000110110100000011100110111: 15.34%
01011100110110011111001101111011100010111110100100: 23.33%
10100100001010100001011011101010110001011010011000: 9.14%
11011011011110000010111100001101000001110111010010: 9.13%
00011000010011000001011111001110011110001000011100: 7.33%
10


# (1 + 1) ES

In [50]:
def mutate(individual):
    mutation_point = randint(0, len(individual) - 1)
    individual[mutation_point] = 1 - individual[mutation_point]  # Flip 0 to 1 and vice versa
    return individual

In [51]:
# (1+1) ES
def one_plus_one_es(problem, iterations):
    genome_length = 1000
    best_individual = choices([0, 1], k=genome_length)
    best_fitness = problem(best_individual)
    
    for _ in range(iterations):
        mutated_individual = mutate(best_individual.copy())
        mutated_fitness = problem(mutated_individual)

        if mutated_fitness >= best_fitness:
            best_individual = mutated_individual
            best_fitness = mutated_fitness

    return best_individual, best_fitness

In [53]:
#test (1+1+) ES:
problem_instance = lab9_lib.make_problem(5)  # Replace 1 with 2, 5, or 10 
iterations = 10000  # Adjust the number of iterations as needed

best_solution, best_fitness = one_plus_one_es(problem_instance, iterations)

print(f"Best solution: {''.join(map(str, best_solution))}")
print(f"Best fitness: {best_fitness:.2%}")
print(f"Total fitness calls: {problem_instance.calls}")

Best solution: 0100100001010000100101001010000000101001010010000000001010010100001001000000100100001000000000100001000000000001000000000100000000000010100001001000010000101001000010100100001010000100001000010010100000000010000000001000000010100000001000000100000001010010100101001000000000101000000010000100000010010100000001000010100000000000000100000000010010100001000010010100000000010010100001000000000000100001010000100000000000000100100001000000100101001000000000100000010000000101000010010100001000010010000101000010000100101001000000100101001010000100001001000010100100000010000100100000010000000100001010010100001001000010000101001010010100100000010010000101001000010000101000000010100001000000010000000001000000100001000000010100000000010000000100000010010000100001010000100100001010010000100000000010000101001010010100100001000000000001001010000100000001000000100101001010000000100001000000000001000010000100100001010010100100001010000000001001010000000100001010010100001001010000000000001

# Mu + lambda ES

In [54]:
# (mu+lamda) ES
def mu_plus_lambda_es(problem, mu, lmbda, generations):
    genome_length = 1000
    population = [choices([0, 1], k=genome_length) for _ in range(mu)]
    
    for generation in range(generations):
        offspring = generate_offspring(population, lmbda)
        combined_population = population + offspring

        combined_population.sort(key=lambda ind: -problem(ind))

        population = combined_population[:mu]

    best_individual = max(population, key=problem)
    best_fitness = problem(best_individual)

    return best_individual, best_fitness

def generate_offspring(parents, lmbda):
    offspring = []
    for _ in range(lmbda):
        parent = choices(parents)[0]
        mutated_individual = mutate(parent.copy())
        offspring.append(mutated_individual)

    return offspring

def mutate(individual):
    mutation_point = randint(0, len(individual) - 1)
    individual[mutation_point] = 1 - individual[mutation_point]  # Flip 0 to 1 and vice versa
    return individual

In [58]:
#test (mu+lamda) ES
problem_instance = lab9_lib.make_problem(1)  # You can replace 1 with 2, 5, or 10 for different instances
mu = 5
lmbda = 20
generations = 1000

best_solution, best_fitness = mu_plus_lambda_es(problem_instance, mu, lmbda, generations)

print(f"Best solution: {''.join(map(str, best_solution))}")
print(f"Best fitness: {best_fitness:.2%}")
print(f"Total fitness calls: {problem_instance.calls}")

Best solution: 1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110111111111111111111101111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111